# Academy of Py School Data Analysis

In [76]:
# Dependencies and Setup
import pandas as pd

# Files to Load
school_data_to_load = "../Input/schools_complete.csv"
student_data_to_load = "../Input/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [77]:
# Combine the data into a single dataset
# We use inner join here as we are not interested neither in the schools with no students nor in the students 
# that don't belong to any school
data = pd.merge(student_data, school_data, how="inner", on=["school_name"]).rename(columns={'school_name' : 'School Name'})
data.head()

,Student ID,student_name,gender,grade,School Name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [78]:
# Lets first check if there are any N/A values
value_counts = data.count()
value_counts

Student ID       39170
student_name     39170
gender           39170
grade            39170
School Name      39170
reading_score    39170
math_score       39170
School ID        39170
type             39170
size             39170
budget           39170
dtype: int64

## District Summary

In [79]:
# Lets get some totals first and merge into a single dataframe
pass_score = 70

school_count = school_data['School ID'].count()
student_count = student_data['Student ID'].count()
total_budget = school_data['budget'].sum()

avg_math_score = student_data['math_score'].mean()
avg_reading_score = student_data['reading_score'].mean()
avg_overal_score = (avg_math_score + avg_reading_score) / 2

pass_math_rate = student_data.loc[student_data['math_score'] >= pass_score]['Student ID'].count() / student_count
pass_reading_rate = student_data.loc[student_data['reading_score'] >= pass_score]['Student ID'].count() / student_count
pass_overal_rate = (pass_math_rate + pass_reading_rate) / 2

district_summary = pd.DataFrame({
    'Total Schools' : school_count,
    'Total Students': f'{student_count:,d}',
    'Total Budget': f'${total_budget:,.2f}',
    'Avg. Math Score': f'{avg_math_score:.2f}',
    'Avg. Reading Score': f'{avg_reading_score:.2f}',
    'Avg. Overall Score': f'{avg_overal_score:.2f}',
    '% Passing Math': f'{pass_math_rate:.2%}',
    '% Passing Reading': f'{pass_reading_rate:.2%}',
    '% Passing Overall': f'{pass_overal_rate:.2%}'
}, index=[0])
district_summary

,Total Schools,Total Students,Total Budget,Avg. Math Score,Avg. Reading Score,Avg. Overall Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428.00",78.99,81.88,80.43,74.98%,85.81%,80.39%


## School Summary

In [80]:
school_groups = data.groupby('School Name')
sc_sum_type = school_groups['type'].first().rename('School Type')
sc_sum_stud_count = school_groups['Student ID'].count().rename('Total Students')
sc_sum_budget = school_groups['budget'].first().rename('Total School Budget')
sc_sum_stud_budget = (sc_sum_budget / sc_sum_stud_count).rename('Per Student Budget')
sc_sum_avg_math = school_groups['math_score'].mean().round(2).rename('Avg. Math Score')
sc_sum_avg_reading = school_groups['reading_score'].mean().round(2).rename('Avg. Reading Score')
sc_sum_pass_math = (school_groups['math_score'].apply(lambda x: (x >= pass_score).sum()) / sc_sum_stud_count) \
.rename('% Passing Math')
sc_sum_pass_reading = (school_groups['reading_score'].apply(lambda x: (x >= pass_score).sum()) / sc_sum_stud_count) \
.rename('% Passing Reading')
sc_sum_pass_overal = ((sc_sum_pass_math + sc_sum_pass_reading) / 2).rename('% Passing Overall')

# Concatenate everything and format everything except overall pass rate (we'll do sorting later based on this one)
school_summary = pd.concat([
    sc_sum_type,
    sc_sum_stud_count.apply('{:,d}'.format),
    sc_sum_budget.apply('${:,.2f}'.format),
    sc_sum_stud_budget.apply('${:.2f}'.format),
    sc_sum_avg_math,
    sc_sum_avg_reading,
    sc_sum_pass_math.apply('{:.2%}'.format),
    sc_sum_pass_reading.apply('{:.2%}'.format),
    sc_sum_pass_overal
], axis=1)
# Now sort everything by overall pass rate and apply format to it
school_summary = school_summary.sort_values('% Passing Overall', ascending=False)
school_summary.loc[:, '% Passing Overall'] = school_summary['% Passing Overall'].apply('{:.2%}'.format)

## Top Performing Schools (By Passing Rate)

In [81]:
school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

In [82]:
school_summary.tail()[::-1]

,School Type,Total Students,Total School Budget,Per Student Budget,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

In [83]:
grades = [str(grade) + 'th' for grade in range(9, 13)]
grades_data = [data.loc[data['grade'] == grade].groupby('School Name') for grade in grades]
math_series = [grade_data['math_score'].mean().round(2).rename(grade) for grade,grade_data in zip(grades, grades_data)]
math_score_by_grades = pd.concat(math_series, axis=1)
math_score_by_grades

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

In [84]:
reading_series = [grade_data['reading_score'].mean().round(2).rename(grade) for grade,grade_data in zip(grades, grades_data)]
reading_score_by_grades = pd.concat(reading_series, axis=1)
reading_score_by_grades

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [85]:
# First lets check what are the min and max 
min_budget_per_student = sc_sum_stud_budget.min()
max_budget_per_student = sc_sum_stud_budget.max()
print(f'The budget per student varies between ${min_budget_per_student} and ${max_budget_per_student}')

The budget per student varies between $578.0 and $655.0


In [86]:
# These will be good groups to split the data
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
# First lets rebuild the summary 
school_summary_raw = pd.concat([
    sc_sum_avg_math,
    sc_sum_avg_reading,
    sc_sum_pass_math,
    sc_sum_pass_reading,
    sc_sum_pass_overal,
    pd.cut(sc_sum_stud_budget, spending_bins, labels=group_names).rename('Spending Ranges (Per Student)')
], axis=1)
# Perform the calculations and necessary formatting
summary_by_per_student_budget = school_summary_raw.groupby('Spending Ranges (Per Student)').mean() 

summary_by_per_student_budget.loc[:, ['Avg. Math Score', 'Avg. Reading Score']] = \
    summary_by_per_student_budget.loc[:, ['Avg. Math Score', 'Avg. Reading Score']].round(2)

summary_by_per_student_budget.loc[:, ['% Passing Math', '% Passing Reading', '% Passing Overall']] = \
    summary_by_per_student_budget.loc[:, ['% Passing Math', '% Passing Reading', '% Passing Overall']] \
    .applymap('{:.2%}'.format)
summary_by_per_student_budget

,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.45,83.94,93.46%,96.61%,95.04%
$585-615,83.60,83.88,94.23%,95.90%,95.07%
$615-645,79.08,81.89,75.67%,86.11%,80.89%
$645-675,77.00,81.03,66.16%,81.13%,73.65%


## Scores by School Size

In [87]:
# First lets check what are the min and max 
min_student_count = sc_sum_stud_count.min()
max_student_count = sc_sum_stud_count.max()
print(f'The school size varies between {min_student_count} and {max_student_count}')

The school size varies between 427 and 4976


In [88]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
# First lets rebuild the summary 
school_summary_raw = pd.concat([
    sc_sum_avg_math,
    sc_sum_avg_reading,
    sc_sum_pass_math,
    sc_sum_pass_reading,
    sc_sum_pass_overal,
    pd.cut(sc_sum_stud_count, size_bins, labels=group_names).rename('School Size')
], axis=1)
school_summary_raw
# Perform the calculations and necessary formatting
summary_by_school_size = school_summary_raw.groupby('School Size').mean() 

summary_by_school_size.loc[:, ['Avg. Math Score', 'Avg. Reading Score']] = \
    summary_by_school_size.loc[:, ['Avg. Math Score', 'Avg. Reading Score']].round(2)

summary_by_school_size.loc[:, ['% Passing Math', '% Passing Reading', '% Passing Overall']] = \
    summary_by_school_size.loc[:, ['% Passing Math', '% Passing Reading', '% Passing Overall']] \
    .applymap('{:.2%}'.format)
summary_by_school_size

,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.87,93.60%,96.79%,95.20%
Large (2000-5000),77.74,81.34,69.96%,82.77%,76.36%


## Scores by School Type

In [89]:
# First lets rebuild the summary 
school_summary_raw = pd.concat([
    sc_sum_avg_math,
    sc_sum_avg_reading,
    sc_sum_pass_math,
    sc_sum_pass_reading,
    sc_sum_pass_overal,
    sc_sum_type
], axis=1)
school_summary_raw
# Perform the calculations and necessary formatting
summary_by_school_type = school_summary_raw.groupby('School Type').mean() 

summary_by_school_type.loc[:, ['Avg. Math Score', 'Avg. Reading Score']] = \
    summary_by_school_type.loc[:, ['Avg. Math Score', 'Avg. Reading Score']].round(2)

summary_by_school_type.loc[:, ['% Passing Math', '% Passing Reading', '% Passing Overall']] = \
    summary_by_school_type.loc[:, ['% Passing Math', '% Passing Reading', '% Passing Overall']] \
    .applymap('{:.2%}'.format)
summary_by_school_type

,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%


## Conclusions

- Looking at scores by school spending we could say that there is a reverse correlation between how much schools spend per student and their academic performance (higher spending corresponds to poorer performance)

- The same can be said about school size. Students of the larger schools in general have poorer performance

- Even though charter schools clearly outperform the district ones (95% comparing to 73% of the overall pass rate) the two abovementioned correlations don't let us make this conclusion based only on school type (e.g. charter schools in general tend to have less students which may contribute to its success)